In [6]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

In [7]:
import pandas as pd
import os
import sys
import json


In [8]:
sys.path.insert(0,os.path.dirname('../src/'))

In [9]:
E1_template = pd.read_csv("../data/E1.csv")

In [10]:
E1_template.head(2)

,AOI,Country,Region,City,Site Name,Description,Method,EO Sensor,Input Data,Indicator code,...,Reference value,Rule,Indicator Value,Sub-AOI,Y axis,Indicator Name,Color code,Data Provider,AOI_ID,Update Frequency
0,"44.408142,8.885851",IT,NaN,Genoa,Port of Genoa and surrounding industrial areas,Import/production sites: status of metallic ores,Object detection on VHR data as proxy for acti...,Planetscope,[NEW] Planetscope COVID-19,E1,...,3,X is the Measurement value. If X<(ref_value-3...,Normal,"MULTIPOLYGON(((8.863167 44.404538,8.86803 44....",Number of ships at piers,Changes in Ships traffic at dedicated piers,BLUE,EMSS,IT3,Weekly
1,"44.408142,8.885851",IT,NaN,Genoa,Port of Genoa and surrounding industrial areas,Import/production sites: status of metallic ores,Object detection on VHR data as proxy for acti...,Planetscope,[NEW] Planetscope COVID-19,E1,...,3,X is the Measurement value. If X<(ref_value-3...,Low,"MULTIPOLYGON(((8.863167 44.404538,8.86803 44....",Number of ships at piers,Changes in Ships traffic at dedicated piers,RED,EMSS,IT3,Weekly


In [35]:
E1_csv_output = pd.DataFrame(columns = E1_template.columns)

In [36]:
E1_csv_output.columns

Index(['AOI', 'Country', 'Region', 'City', 'Site Name', 'Description',
       'Method', 'EO Sensor', 'Input Data', 'Indicator code', 'Time',
       'Measurement Value', 'Reference Description', 'Reference time',
       'Reference value', 'Rule', 'Indicator Value', 'Sub-AOI', 'Y axis',
       'Indicator Name', 'Color code', 'Data Provider', 'AOI_ID',
       'Update Frequency'],
      dtype='object')

In [13]:
with open('../data/aoi.json', 'r') as f:
    aoi_file = json.load(f)

interest = 'Final'

In [37]:
def get_color_code(date_time_str):
    dt_obj = datetime.strptime(date_time_str, '%Y-%m-%d')
    if dt_obj.year == 2020:
        return "RED"
    else:
        return "GREEN"
    
sitename_to_countrycode = {
        "Sines" : "PT",
        "Vigo" : "ES",
        "Antwerp" : "BE",
        "Budapest" : "HU" ,
        "Rotterdam" : "NL",
        "Kiel Canal": "DE",
        "Grand Harbour" : "MT",
        "Dublin" : "IE",
        "Tallin-Helsinki": "FI",
        "Limassol" : "CY",
        "Athen Pireaus" : "GR",
        "Bosporus" : "TR",
        "Dardanelles" : "TR",
        "Gibraltar" :"GB",
        "Dover" : "GB",
        "Corfu" : "GR",
        "Messina": "IT",
        "Oresund": "SE",
        "Venice" : "IT",
        "St Tropez" :"FR",
        "Split" : "HR"
    }
sitename_to_city = {
        "Sines" : "Sines",
        "Vigo" : "Vigo",
        "Antwerp" : "Antwerp",
        "Budapest" : "Budapest" ,
        "Rotterdam" : "Rotterdam",
        "Kiel Canal": "Kiel",
        "Grand Harbour" : "Valleta",
        "Dublin" : "Dublin",
        "Tallin-Helsinki": "Helsinki",
        "Limassol" : "Limassol",
        "Athen Pireaus" : "Athen",
        "Bosporus" : "Bosporus",
        "Dardanelles" : "Dardanelles",
        "Gibraltar" :"Gibraltar",
        "Dover" : "Dover",
        "Corfu" : "Corfu",
        "Messina": "Messina",
        "Oresund": "Oresund",
        "Venice" : "Venice",
        "St Tropez" :"St Tropez",
        "Split" : "Split"
    }
  

In [38]:
radius=7000
version="0.1.1"
for sitename, lat_lons in aoi_file[interest].items():
    for lat_lon in lat_lons:
        output_file = '{}_lat={}_lon={}_r={}_v={}.json'.format(sitename.lower(), lat_lon[0], lat_lon[1], radius, version)
        output_full_path = os.path.join("../data/outputs", output_file)
        if os.path.isfile(output_full_path):
            f = open(output_full_path)
            output_obj = json.load(f)
            for time, count in output_obj.items():
#                 print(f"time: {time}, count: {count}")
                AOI = f"{lat_lon[0]},{lat_lon[1]}"
                Country = sitename_to_countrycode[sitename]
                City = sitename_to_city[sitename]
                Site_name = sitename
                Description = ""
                Method = "Object Counting"
                EO_sensor = "Sentinel-2"
                Input_data = "Sentinel 2 L1C"
                Indicator_code = "E1"
                Time = time
                Measurement_value = count
                Reference_description = f"Number of ships in {2 * radius / 1000} km x {2 * radius / 1000} km area"
                Reference_time = time
                Reference_value = ""
                Rule = ""
                Indicator_Value = ""
                Sub_AOI = ""
                Y_axis = "Number of ships inferred"
                Indicator_Name = "Changes in Ships traffic"
                Color_code = get_color_code(time)
                Data_Provider = ""
                AOI_ID = ""
                Update_Frequency = "Weekly"
                row = {'AOI':AOI, 'Country':Country, 'Region':"", 'City':City, 'Site Name':Site_name, 'Description':Description,
                       'Method':Method, 'EO Sensor':EO_sensor, 'Input Data':Input_data, 'Indicator code':Indicator_code, 'Time':Time,
                       'Measurement Value':Measurement_value, 'Reference Description':Reference_description, 'Reference time': Reference_time,
                       'Reference value':Reference_value, 'Rule':Rule, 'Indicator Value':Indicator_Value, 'Sub-AOI':Sub_AOI, 'Y axis':Y_axis,
                       'Indicator Name':Indicator_Name, 'Color code':Color_code, 'Data Provider':Data_Provider, 'AOI_ID':AOI_ID,
                       'Update Frequency':Update_Frequency}
                E1_csv_output = E1_csv_output.append(row, ignore_index=True)
            

In [39]:
E1_csv_output.head(3)

,AOI,Country,Region,City,Site Name,Description,Method,EO Sensor,Input Data,Indicator code,...,Reference value,Rule,Indicator Value,Sub-AOI,Y axis,Indicator Name,Color code,Data Provider,AOI_ID,Update Frequency
0,"37.88,-8.91",PT,,Sines,Sines,,Object Counting,Sentinel-2,Sentinel 2 L1C,E1,...,,,,,Number of ships inferred,Changes in Ships traffic,GREEN,,,Weekly
1,"37.88,-8.91",PT,,Sines,Sines,,Object Counting,Sentinel-2,Sentinel 2 L1C,E1,...,,,,,Number of ships inferred,Changes in Ships traffic,GREEN,,,Weekly
2,"37.88,-8.91",PT,,Sines,Sines,,Object Counting,Sentinel-2,Sentinel 2 L1C,E1,...,,,,,Number of ships inferred,Changes in Ships traffic,GREEN,,,Weekly


In [40]:
E1_csv_output.to_csv(f"../data/E1_{datetime.now()}.csv")

In [2]:

def get_measurement_value_2020(output_obj):
    pass

def get_reference_time_2019_2018(output_obj):
    pass

# output_obj is the json object
def get_reference_value_2019_2018(output_obj):
    for time, count in output_obj.items():
        pass

def get_color():
    pass

def get_indicator_value():
    pass
    

In [4]:
date_time_str = "2020-01-01"
dt_obj = datetime.strptime(date_time_str, '%Y-%m-%d')
dt_obj

datetime.datetime(2020, 1, 1, 0, 0)

In [5]:
def get_color_code(date_time_str):
    dt_obj = datetime.strptime(date_time_str, '%Y-%m-%d')
    if dt_obj.year == 2020:
        return "RED"
    elif:
        return "GREEN"

2020

In [33]:
print(datetime.now().date())

2020-07-11
